In [1]:
import numpy as np
import tensorflow as tf

latest = tf.train.latest_checkpoint("")
latest

g = tf.Graph()
with g.as_default():
    saver = tf.train.import_meta_graph('model_200.ckpt.meta')

sess = tf.Session(graph = g)

with sess.as_default():
    saver.restore(sess, 'model_200.ckpt')
    

c:\users\wiktor\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\wiktor\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\wiktor\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\wiktor\

In [2]:
[op.name for op in g.get_operations()]

['tf_x',
 'tf_x_image/shape',
 'tf_x_image',
 'tf_y',
 'tf_y_onehot/on_value',
 'tf_y_onehot/off_value',
 'tf_y_onehot/depth',
 'tf_y_onehot',
 'is_train',
 'layer1/kernel/Initializer/random_uniform/shape',
 'layer1/kernel/Initializer/random_uniform/min',
 'layer1/kernel/Initializer/random_uniform/max',
 'layer1/kernel/Initializer/random_uniform/RandomUniform',
 'layer1/kernel/Initializer/random_uniform/sub',
 'layer1/kernel/Initializer/random_uniform/mul',
 'layer1/kernel/Initializer/random_uniform',
 'layer1/kernel',
 'layer1/kernel/Assign',
 'layer1/kernel/read',
 'layer1/bias/Initializer/zeros',
 'layer1/bias',
 'layer1/bias/Assign',
 'layer1/bias/read',
 'layer1/dilation_rate',
 'layer1/Conv2D',
 'layer1/BiasAdd',
 'layer1/Sigmoid',
 'layer1_pool/MaxPool',
 'layer2/kernel/Initializer/random_uniform/shape',
 'layer2/kernel/Initializer/random_uniform/min',
 'layer2/kernel/Initializer/random_uniform/max',
 'layer2/kernel/Initializer/random_uniform/RandomUniform',
 'layer2/kernel/Init

In [3]:
from scipy.io import wavfile
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
fs, data = wavfile.read('voive_sample.wav')

print(data.shape, fs)
plt.plot(data)
plt.show()
data = data[10000:26000]
print(data.shape)

f, t, Sxx = spectrogram(data, window='hamming')
plt.imshow(Sxx)
plt.show()

sigma = np.std(Sxx)
X_data = Sxx / sigma
print(np.min(X_data))
print(np.max(X_data))
print(X_data.shape)

print(X_data.shape)
X_image = X_data[:70, :70]
plt.imshow(X_image)
plt.show()
X_image = np.expand_dims(X_image, axis=0)
print(X_image.shape)


X_input = np.reshape(X_image, (-1, 4900))
label_names = ['no', 'yes', 'on', 'off', 'down', 'up', 'left', 'right', 'go', 'wow']

(57301,) 16000


c:\users\wiktor\appdata\local\programs\python\python36\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


<Figure size 640x480 with 1 Axes>

(16000,)


<Figure size 640x480 with 1 Axes>

5.4467778e-11
44.78458
(129, 71)
(129, 71)


<Figure size 640x480 with 1 Axes>

(1, 70, 70)


In [5]:
import timeit

In [6]:
start = timeit.timeit()
with sess.as_default():
    feed = {'tf_x:0' : X_input, 'is_train:0' : False}
    y_pred = sess.run(['predicted_labels:0'], feed_dict=feed)

y_pred = np.array(y_pred)
idx = y_pred[0][0]
print(idx)

print(label_names[idx])
end = timeit.timeit()
print('time elapsed')
print (end - start)

0
no
time elapsed
-0.015742200000000005


In [7]:
from recognize import recognize
print(recognize())

TypeError: recognize() missing 1 required positional argument: 'data'

In [25]:
import pyaudio
import wave
import sys
from recognize import recognize
import matplotlib.pyplot as plt
from tqdm import tqdm

In [42]:
RATE    = 16000
CHUNK   = 16000
skipnum = 8000
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16,
                channels=2,
                rate=RATE,
                output=True)       #stream wyjsciowy (odtwarzany na sluchawkach)
mic_stream = p.open(format=pyaudio.paInt16,
                channels=2,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)#dane wejsciowe ( z mikrofonu)
# sformatowanie danych do postaci tabeli liczb typu int16(2 bajty)
               # zapisanie pierwszego kanalu
i = 10
while i>0:
    i -= 1
    print("speak", end=": ")
    stream.write(mic_stream.read(CHUNK))        # odtworzenie zapisanych danych
#    data = audio_file.readframes(CHUNK)
    # dla odczytu z pliku
    data = mic_stream.read(CHUNK)               # wczytanie danych z mikrofonu 
    data = np.frombuffer(data,dtype=np.int) # sformatowanie kolejnej porcji danych
    try:											# przechwycenie wyjatko
        print(recognize(data))

    except ValueError:
        print("data ends")
    


speak: down
speak: off
speak: off
speak: off
speak: wow
speak: go
speak: off
speak: off
speak: off
speak: on


In [7]:

if __name__ == "__main__":
    (recognize())

W1003 22:01:17.358400  6600 deprecation_wrapper.py:119] From C:\Users\Wiktor\Desktop\Programowanie\Kaggle_Competitions\Speech_Recognition\recognize.py:11: The name tf.train.import_meta_graph is deprecated. Please use tf.compat.v1.train.import_meta_graph instead.

W1003 22:01:17.508383  6600 deprecation_wrapper.py:119] From C:\Users\Wiktor\Desktop\Programowanie\Kaggle_Competitions\Speech_Recognition\recognize.py:13: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

c:\users\wiktor\appdata\local\programs\python\python36\lib\site-packages\scipy\signal\spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 2, using nperseg = 2
  .format(nperseg, input_length))


ValueError: cannot reshape array of size 140 into shape (4900)